# ScaNN
---
## Download Dataset
---

In [4]:
%pip install numpy h5py requests scann

  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-core 2.10.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.4.2 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# import libraries
import numpy as np
import h5py
import os
import requests
import tempfile
import time

import scann

2023-10-14 09:17:22.968497: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-14 09:17:23.205637: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-14 09:17:23.206704: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 09:17:24.190828: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
with tempfile.TemporaryDirectory() as tmp:
    response = requests.get("http://ann-benchmarks.com/sift-128-euclidean.hdf5")
    loc = os.path.join(tmp, "sift.hdf5")
    with open(loc, 'wb') as f:
        f.write(response.content)
    
    sift_h5py = h5py.File(loc, "r")

In [7]:
list(sift_h5py.keys())

['distances', 'neighbors', 'test', 'train']

In [8]:
dataset = sift_h5py['train']
queries = sift_h5py['test']
print(dataset.shape)
print(queries.shape)

(1000000, 128)
(10000, 128)


## Create a ScaNN Searcher
--- 

In [9]:
normalized_dataset = dataset / np.linalg.norm(dataset, axis=1)[:, np.newaxis]
# configure ScaNN as a tree - asymmetric hash hybrid with reordering
# anisotropic quantization as described in the paper; see README

# use scann.scann_ops.build() to instead create a TensorFlow-compatible searcher
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

2023-10-14 09:20:49.496407: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 249544
2023-10-14 09:20:55.523612: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:84] PartitionerFactory ran in 6.027129817s.


In [10]:
def compute_recall(neighbors, true_neighbors):
    total = 0
    for gt_row, row in zip(true_neighbors, neighbors):
        total += np.intersect1d(gt_row, row).shape[0]
    return total / true_neighbors.size

## ScaNN interface features
---

In [11]:
# this will search the top 100 of the 2000 leaves, and compute
# the exact dot products of the top 100 candidates from asymmetric
# hashing to get the final top 10 candidates.
start = time.time()
neighbors, distances = searcher.search_batched(queries)
end = time.time()

# we are given top 100 neighbors in the ground truth, so select top 10
print("Recall:", compute_recall(neighbors, sift_h5py['neighbors'][:, :10]))
print("Time:", end - start)

Recall: 0.98078
Time: 1.257188320159912


In [12]:
# increasing the leaves to search increases recall at the cost of speed
start = time.time()
neighbors, distances = searcher.search_batched(queries, leaves_to_search=150)
end = time.time()

print("Recall:", compute_recall(neighbors, sift_h5py['neighbors'][:, :10]))
print("Time:", end - start)

Recall: 0.98533
Time: 1.6079840660095215


In [13]:
# increasing reordering (the exact scoring of top AH candidates) has a similar effect.
start = time.time()
neighbors, distances = searcher.search_batched(queries, leaves_to_search=150, pre_reorder_num_neighbors=250)
end = time.time()

print("Recall:", compute_recall(neighbors, sift_h5py['neighbors'][:, :10]))
print("Time:", end - start)

Recall: 0.98934
Time: 1.995032548904419


In [14]:
# we can also dynamically configure the number of neighbors returned
# currently returns 10 as configued in ScannBuilder()
neighbors, distances = searcher.search_batched(queries)
print(neighbors.shape, distances.shape)

# now returns 20
neighbors, distances = searcher.search_batched(queries, final_num_neighbors=20)
print(neighbors.shape, distances.shape)

(10000, 10) (10000, 10)
(10000, 20) (10000, 20)


In [15]:
# we have been exclusively calling batch search so far; the single-query call has the same API
start = time.time()
neighbors, distances = searcher.search(queries[0], final_num_neighbors=5)
end = time.time()

print(neighbors)
print(distances)
print("Latency (ms):", 1000*(end - start))

[932085 934876 561813 708177 706771]
[454.80237 453.99243 449.59064 443.9261  443.51404]
Latency (ms): 0.4856586456298828
